<a href="https://colab.research.google.com/github/GUILLENJV/62f1ab79cc287b001f643389/blob/master/Grid_Search_Hyperparameters_MULTICLASS_MODEL_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Grid Search Hyperparameters MULTICLASS MODEL

In [ ]:
# example of learned embedding encoding for a neural network
from numpy import unique
import numpy
from numpy import array
import pandas as pd
import datetime 
from time import sleep
from tqdm import tqdm
import os
from pandas import read_csv
from sklearn.model_selection import train_test_split
import json
import tensorflow as tf
from tensorflow import keras
from keras.utils import np_utils
import requests
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from tensorflow.keras.layers import BatchNormalization
from sklearn.preprocessing import StandardScaler
from keras.regularizers import l1
from keras.regularizers import l2
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.callbacks import EarlyStopping
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import regularizers
from keras.utils.vis_utils import plot_model
from matplotlib import pyplot
import matplotlib.pyplot as plt
import seaborn as sns
# Use scikit-learn to grid search the batch size and epochs
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

In [ ]:
# carga los datos
df = pd.read_csv('631d8277776f1d001a21e6eb.csv') 
df = df.drop_duplicates()

df["visitor_device_hardware_family"] = df["visitor_device_hardware_family"].apply(str)
df["visitor_device_hardware_model"] = df["visitor_device_hardware_model"].apply(str)
df["visitor_device_os_version"] = df["visitor_device_os_version"].apply(str)
df["visitor_device_hardware_family"] = df["visitor_device_hardware_family"].apply(str)
df["visitor_device_hardware_model"] = df["visitor_device_hardware_model"].apply(str)
df["visitor_device_os_version"] = df["visitor_device_os_version"].apply(str)
df["visitor_tokens_adh"] = df["visitor_tokens_adh"].apply(str)
df["visitor_tokens_cadid"] = df["visitor_tokens_cadid"].apply(str)
df["converted_yes"] = df["converted_yes"].apply(str)
df["converted_no"] = df["converted_no"].apply(str)

# Imputamos la variable 
df['visitor_device_os_vendor'].fillna(df['visitor_device_os_vendor'].mode()[0], inplace=True)
df['campaign_id'].fillna(df['campaign_id'].mode()[0], inplace=True)
df['visitor_device_hardware_vendor'].fillna(df['visitor_device_hardware_vendor'].mode()[0], inplace=True)
df['visitor_device_browser'].fillna(df['visitor_device_browser'].mode()[0], inplace=True)
df['traffic_source_id'].fillna(df['traffic_source_id'].mode()[0], inplace=True)
df['landing_page_id'].fillna(df['landing_page_id'].mode()[0], inplace=True)
df['visitor_device_os_family'].fillna(df['visitor_device_os_family'].mode()[0], inplace=True)
df['visitor_device_type'].fillna(df['visitor_device_type'].mode()[0], inplace=True)
df['visitor_geo_location_countryCode'].fillna(df['visitor_geo_location_countryCode'].mode()[0], inplace=True)
df['visitor_geo_location_regionName'].fillna(df['visitor_geo_location_regionName'].mode()[0], inplace=True)
df['visitor_tokens_adi'].fillna(df['visitor_tokens_adi'].mode()[0], inplace=True)
df['visitor_geo_location_isp'].fillna(df['visitor_geo_location_isp'].mode()[0], inplace=True)
df['visitor_geo_location_connection_type'].fillna(df['visitor_geo_location_connection_type'].mode()[0], inplace=True)
df['visitor_geo_location_cityName'].fillna(df['visitor_geo_location_cityName'].mode()[0], inplace=True)

df = df.drop(['landing_pages_group_id','visitor_tokens_add'], axis=1)

X = df[['campaign_id', 'traffic_source_id',
       'visitor_device_browser', 'visitor_device_hardware_family',
       'visitor_device_hardware_model', 'visitor_device_hardware_vendor',
       'visitor_device_os_family', 'visitor_device_os_vendor',
       'visitor_device_os_version', 'visitor_device_type',
       'visitor_geo_location_cityName', 'visitor_geo_location_connection_type',
       'visitor_geo_location_countryCode', 'visitor_geo_location_isp',
       'visitor_geo_location_regionName', 'visitor_tokens_adh',
       'visitor_tokens_cadid', 'visitor_tokens_adi', 'converted_yes',
       'converted_no']]

X = X.astype(str).to_numpy()
y = df[["landing_page_id"]].to_numpy()

# encode string input values as integers
encoded_x = None
for i in range(0, X.shape[1]):
    label_encoder = LabelEncoder()
    feature = label_encoder.fit_transform(X[:,i])
    feature = feature.reshape(X.shape[0], 1)
    onehot_encoder = OrdinalEncoder()
    feature = onehot_encoder.fit_transform(feature)
    if encoded_x is None:
        encoded_x = feature
    else:
        encoded_x = numpy.concatenate((encoded_x, feature), axis=1)
#print("X shape: : ", encoded_x.shape)

# encode string class values as integers
label_encoder = LabelEncoder()
label_encoder = label_encoder.fit(y)
label_encoded_y = label_encoder.transform(y)
dummy_y = np_utils.to_categorical(label_encoded_y)

# split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(encoded_x, dummy_y, test_size=0.30, random_state=123)

scaler=StandardScaler()
scaler.fit(X_train)
X_train=scaler.transform(X_train)
X_test=scaler.transform(X_test)

# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


# 1. Batch Size and Number of Epochs

In [ ]:
# Definición del modelo
def modelo_mlp(): 
    model = Sequential()
    model.add(Dense(20, input_dim=X_train.shape[1], 
                kernel_initializer= 'he_uniform',
                activation='relu', activity_regularizer=regularizers.l1(1e-4)))
    model.add(BatchNormalization())
    model.add(Dense(10, kernel_initializer= 'he_uniform',
                activation='relu',activity_regularizer=regularizers.l1(1e-4)))
    model.add(Dropout(0.3))
    model.add(BatchNormalization())
    model.add(Dense(5, kernel_initializer= 'he_uniform',
                activation='relu',activity_regularizer=regularizers.l1(1e-4)))
    model.add(Dropout(0.3))
    model.add(BatchNormalization())  
    model.add(Dense(3, kernel_initializer= 'he_uniform',
                activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam',
                metrics=['accuracy'])
    model.summary()
    return model

In [ ]:
# create model
model = KerasClassifier(build_fn=modelo_mlp, verbose=0)
# define the grid search parameters
batch_size = [32, 64, 128]
epochs = [10, 50, 100]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 20)                420       
                                                                 
 batch_normalization (BatchN  (None, 20)               80        
 ormalization)                                                   
                                                                 
 dense_1 (Dense)             (None, 10)                210       
                                                                 
 dropout (Dropout)           (None, 10)                0         
                                                                 
 batch_normalization_1 (Batc  (None, 10)               40        
 hNormalization)                                                 
                                                                 
 dense_2 (Dense)             (None, 5)                 5

In [ ]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.634477 using {'batch_size': 64, 'epochs': 100}
0.598941 (0.014190) with: {'batch_size': 32, 'epochs': 10}
0.631013 (0.004423) with: {'batch_size': 32, 'epochs': 50}
0.634477 (0.007368) with: {'batch_size': 32, 'epochs': 100}
0.560860 (0.005970) with: {'batch_size': 64, 'epochs': 10}
0.610242 (0.010985) with: {'batch_size': 64, 'epochs': 50}
0.634477 (0.009753) with: {'batch_size': 64, 'epochs': 100}
0.556675 (0.024290) with: {'batch_size': 128, 'epochs': 10}
0.626278 (0.009530) with: {'batch_size': 128, 'epochs': 50}
0.634476 (0.003692) with: {'batch_size': 128, 'epochs': 100}


#2. Training Optimization Algorithm

In [ ]:
# Definición del modelo
def modelo_mlp(optimizer='adam'): 
    model = Sequential()
    model.add(Dense(20, input_dim=X_train.shape[1], 
                kernel_initializer= 'he_uniform',
                activation='relu', activity_regularizer=regularizers.l1(1e-4)))
    model.add(BatchNormalization())
    model.add(Dense(10, kernel_initializer= 'he_uniform',
                activation='relu',activity_regularizer=regularizers.l1(1e-4)))
    model.add(Dropout(0.3))
    model.add(BatchNormalization())
    model.add(Dense(5, kernel_initializer= 'he_uniform',
                activation='relu',activity_regularizer=regularizers.l1(1e-4)))
    model.add(Dropout(0.3))
    model.add(BatchNormalization())  
    model.add(Dense(3, kernel_initializer= 'he_uniform',
                activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam',
                metrics=['accuracy'])
    model.summary()
    return model

In [ ]:
# create model
model = KerasClassifier(build_fn=modelo_mlp, epochs=100, batch_size=32, verbose=0)
# define the grid search parameters
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
param_grid = dict(optimizer=optimizer)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 20)                420       
                                                                 
 batch_normalization_6 (Batc  (None, 20)               80        
 hNormalization)                                                 
                                                                 
 dense_9 (Dense)             (None, 10)                210       
                                                                 
 dropout_2 (Dropout)         (None, 10)                0         
                                                                 
 batch_normalization_7 (Batc  (None, 10)               40        
 hNormalization)                                                 
                                                                 
 dense_10 (Dense)            (None, 5)                

In [ ]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.644859 using {'optimizer': 'Adadelta'}
0.637208 (0.005038) with: {'optimizer': 'SGD'}
0.639759 (0.004462) with: {'optimizer': 'RMSprop'}
0.639577 (0.003834) with: {'optimizer': 'Adagrad'}
0.644859 (0.008273) with: {'optimizer': 'Adadelta'}
0.636481 (0.005953) with: {'optimizer': 'Adam'}
0.644677 (0.008022) with: {'optimizer': 'Adamax'}
0.634476 (0.005441) with: {'optimizer': 'Nadam'}


# 3. Learning Rate and Momentum

In [ ]:
# Definición del modelo
def modelo_mlp(learn_rate=0.01, momentum=0): 
    model = Sequential()
    model.add(Dense(20, input_dim=X_train.shape[1], 
                kernel_initializer= 'he_uniform',
                activation='relu', activity_regularizer=regularizers.l1(1e-4)))
    model.add(BatchNormalization())
    model.add(Dense(10, kernel_initializer= 'he_uniform',
                activation='relu',activity_regularizer=regularizers.l1(1e-4)))
    model.add(Dropout(0.3))
    model.add(BatchNormalization())
    model.add(Dense(5, kernel_initializer= 'he_uniform',
                activation='relu',activity_regularizer=regularizers.l1(1e-4)))
    model.add(Dropout(0.3))
    model.add(BatchNormalization())  
    model.add(Dense(3, kernel_initializer= 'he_uniform',
                activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam',
                metrics=['accuracy'])
    model.summary()
    return model

In [ ]:
# create model
model = KerasClassifier(build_fn=modelo_mlp, epochs=100, batch_size=32, verbose=0)
# define the grid search parameters
learn_rate = [0.0001,0.001, 0.01, 0.1]
momentum = [0.0, 0.2, 0.4, 0.6, 0.8]
param_grid = dict(learn_rate=learn_rate, momentum=momentum)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 20)                420       
                                                                 
 batch_normalization_9 (Batc  (None, 20)               80        
 hNormalization)                                                 
                                                                 
 dense_13 (Dense)            (None, 10)                210       
                                                                 
 dropout_4 (Dropout)         (None, 10)                0         
                                                                 
 batch_normalization_10 (Bat  (None, 10)               40        
 chNormalization)                                                
                                                                 
 dense_14 (Dense)            (None, 5)                

In [ ]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.646864 using {'learn_rate': 0.001, 'momentum': 0.0}
0.639395 (0.005134) with: {'learn_rate': 0.0001, 'momentum': 0.0}
0.645223 (0.012156) with: {'learn_rate': 0.0001, 'momentum': 0.2}
0.634477 (0.005724) with: {'learn_rate': 0.0001, 'momentum': 0.4}
0.635569 (0.004947) with: {'learn_rate': 0.0001, 'momentum': 0.6}
0.640303 (0.014025) with: {'learn_rate': 0.0001, 'momentum': 0.8}
0.646864 (0.009479) with: {'learn_rate': 0.001, 'momentum': 0.0}
0.636298 (0.005904) with: {'learn_rate': 0.001, 'momentum': 0.2}
0.635753 (0.009094) with: {'learn_rate': 0.001, 'momentum': 0.4}
0.634841 (0.007540) with: {'learn_rate': 0.001, 'momentum': 0.6}
0.640853 (0.003942) with: {'learn_rate': 0.001, 'momentum': 0.8}
0.639758 (0.005331) with: {'learn_rate': 0.01, 'momentum': 0.0}
0.638120 (0.006034) with: {'learn_rate': 0.01, 'momentum': 0.2}
0.645044 (0.000476) with: {'learn_rate': 0.01, 'momentum': 0.4}
0.645588 (0.010355) with: {'learn_rate': 0.01, 'momentum': 0.6}
0.633565 (0.006991) with: {'l

#4. Network Weight Initialization

In [ ]:
# Definición del modelo
def modelo_mlp(init_mode='uniform'): 
    model = Sequential()
    model.add(Dense(20, input_dim=X_train.shape[1], 
                kernel_initializer= 'he_uniform',
                activation='relu', activity_regularizer=regularizers.l1(1e-4)))
    model.add(BatchNormalization())
    model.add(Dense(10, kernel_initializer= 'he_uniform',
                activation='relu',activity_regularizer=regularizers.l1(1e-4)))
    model.add(Dropout(0.3))
    model.add(BatchNormalization())
    model.add(Dense(5, kernel_initializer= 'he_uniform',
                activation='relu',activity_regularizer=regularizers.l1(1e-4)))
    model.add(Dropout(0.3))
    model.add(BatchNormalization())  
    model.add(Dense(3, kernel_initializer= 'he_uniform',
                activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam',
                metrics=['accuracy'])
    model.summary()
    return model

In [ ]:
# create model
model = KerasClassifier(build_fn=modelo_mlp, epochs=100, batch_size=32, verbose=0)
# define the grid search parameters
init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
param_grid = dict(init_mode=init_mode)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  
/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:703: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_16 (Dense)            (None, 20)                420       
                                                                 
 batch_normalization_12 (Bat  (None, 20)               80        
 chNormalization)                                                
                                                                 
 dense_17 (Dense)            (None, 10)                210       
                                                                 
 dropout_6 (Dropout)         (None, 10)                0         
                                                                 
 batch_normalization_13 (Bat  (None, 10)               40        
 chNormalization)                                                
                                                                 
 dense_18 (Dense)            (None, 5)                

In [ ]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.649231 using {'init_mode': 'zero'}
0.642493 (0.006417) with: {'init_mode': 'uniform'}
0.647957 (0.011682) with: {'init_mode': 'lecun_uniform'}
0.637392 (0.008025) with: {'init_mode': 'normal'}
0.649231 (0.015993) with: {'init_mode': 'zero'}
0.644859 (0.010569) with: {'init_mode': 'glorot_normal'}
0.634113 (0.008910) with: {'init_mode': 'glorot_uniform'}
0.630468 (0.006502) with: {'init_mode': 'he_normal'}
0.633930 (0.007062) with: {'init_mode': 'he_uniform'}


#5. Neuron Activation Function

In [ ]:
# Definición del modelo
def modelo_mlp(activation='relu'): 
    model = Sequential()
    model.add(Dense(20, input_dim=X_train.shape[1], 
                kernel_initializer= 'he_uniform',
                activation='relu', activity_regularizer=regularizers.l1(1e-4)))
    model.add(BatchNormalization())
    model.add(Dense(10, kernel_initializer= 'he_uniform',
                activation='relu',activity_regularizer=regularizers.l1(1e-4)))
    model.add(Dropout(0.3))
    model.add(BatchNormalization())
    model.add(Dense(5, kernel_initializer= 'he_uniform',
                activation='relu',activity_regularizer=regularizers.l1(1e-4)))
    model.add(Dropout(0.3))
    model.add(BatchNormalization())  
    model.add(Dense(3, kernel_initializer= 'he_uniform',
                activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam',
                metrics=['accuracy'])
    model.summary()
    return model

In [ ]:
# create model
model = KerasClassifier(build_fn=modelo_mlp, epochs=100, batch_size=32, verbose=0)
# define the grid search parameters
activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
param_grid = dict(activation=activation)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_20 (Dense)            (None, 20)                420       
                                                                 
 batch_normalization_15 (Bat  (None, 20)               80        
 chNormalization)                                                
                                                                 
 dense_21 (Dense)            (None, 10)                210       
                                                                 
 dropout_8 (Dropout)         (None, 10)                0         
                                                                 
 batch_normalization_16 (Bat  (None, 10)               40        
 chNormalization)                                                
                                                                 
 dense_22 (Dense)            (None, 5)                

In [ ]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.646317 using {'activation': 'tanh'}
0.635753 (0.008203) with: {'activation': 'softmax'}
0.640489 (0.001492) with: {'activation': 'softplus'}
0.633383 (0.004056) with: {'activation': 'softsign'}
0.635934 (0.007548) with: {'activation': 'relu'}
0.646317 (0.012151) with: {'activation': 'tanh'}
0.634294 (0.006370) with: {'activation': 'sigmoid'}
0.634112 (0.006871) with: {'activation': 'hard_sigmoid'}
0.637027 (0.005035) with: {'activation': 'linear'}


#6. Number of Neurons in the Hidden Layer

In [ ]:
# Definición del modelo
def modelo_mlp(neurons=1): 
    model = Sequential()
    model.add(Dense(20, input_dim=X_train.shape[1], 
                kernel_initializer= 'he_uniform',
                activation='relu', activity_regularizer=regularizers.l1(1e-4)))
    model.add(BatchNormalization())
    model.add(Dense(10, kernel_initializer= 'he_uniform',
                activation='relu',activity_regularizer=regularizers.l1(1e-4)))
    model.add(Dropout(0.3))
    model.add(BatchNormalization())
    model.add(Dense(5, kernel_initializer= 'he_uniform',
                activation='relu',activity_regularizer=regularizers.l1(1e-4)))
    model.add(Dropout(0.3))
    model.add(BatchNormalization())  
    model.add(Dense(3, kernel_initializer= 'he_uniform',
                activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam',
                metrics=['accuracy'])
    model.summary()
    return model

In [ ]:
# create model
model = KerasClassifier(build_fn=modelo_mlp, epochs=100, batch_size=32, verbose=0)
# define the grid search parameters
neurons = [1, 3, 5, 10]
param_grid = dict(neurons=neurons)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  


Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_24 (Dense)            (None, 20)                420       
                                                                 
 batch_normalization_18 (Bat  (None, 20)               80        
 chNormalization)                                                
                                                                 
 dense_25 (Dense)            (None, 10)                210       
                                                                 
 dropout_10 (Dropout)        (None, 10)                0         
                                                                 
 batch_normalization_19 (Bat  (None, 10)               40        
 chNormalization)                                                
                                                                 
 dense_26 (Dense)            (None, 5)                

In [ ]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.643401 using {'neurons': 10}
0.637573 (0.006301) with: {'neurons': 1}
0.639031 (0.005163) with: {'neurons': 3}
0.642127 (0.006139) with: {'neurons': 5}
0.643401 (0.010804) with: {'neurons': 10}
